# Importation des bibliothéques

In [2]:
import pandas as pd
import glob

# Lecture des fichiers du dossier BDLE_10K

In [5]:
parquet_folder = "./local/data/BDLE_10K/"

# Lire tous les fichiers Parquet dans une liste de DataFrames
dfs = [pd.read_parquet(fichier) for fichier in glob.glob(parquet_folder + '*.parquet')]

# Concaténer les DataFrames en un seul DataFrame
df = pd.concat(dfs, ignore_index=True)

# Afficher les premières lignes du DataFrame
df.head()

,seqID1,seqID2,sim
0,146014049:1,146875430:4,100.000000
1,152779129:0,153474359:1,94.199997
2,152961550:2,154232440:1,100.000000
3,152935015:0,19084589:5,87.000000
4,152761649:2,51873187:3,92.900002


In [7]:
df.shape

(20842, 3)

# Partionnement des arêtes en plusieurs sous ensembles

In [10]:
# Nombre de partitions
num_partitions = 20

# Ajout d'une nouvelle colonne 'partition' basée sur le modulo de seqID1
df['partition'] = df['seqID1'].apply(lambda x: hash(x) % num_partitions)

# Afficher le DataFrame partitionné
df

,seqID1,seqID2,sim,partition
0,146014049:1,146875430:4,100.000000,3
1,152779129:0,153474359:1,94.199997,5
2,152961550:2,154232440:1,100.000000,11
3,152935015:0,19084589:5,87.000000,4
4,152761649:2,51873187:3,92.900002,16
...,...,...,...,...
20837,116893109:5,117553577:4,96.199997,0
20838,126431726:1,158049846:4,97.400002,11
20839,128905595:1,138274230:0,97.000000,12
20840,117553577:3,METdb_00052-1-DN38204-c2-g2-i1.p1,98.400002,9


#### La taille(nombre d'arêtes) de chaque partition

In [11]:
#La taille(nombre d'arêtes) de chaque partition
partition_freq = df.groupby('partition').size().reset_index(name='count')

# Afficher la fréquence de partition
partition_freq

,partition,count
0,0,931
1,1,1051
2,2,978
3,3,1080
4,4,1139
5,5,1092
6,6,1110
7,7,994
8,8,1057
9,9,984


#### Création un DataFrame pour chaque partition

In [13]:
partitions = partition_freq['partition'].unique()
for partition in partitions:
    df_partition = df[df['partition'] == partition]
    file_path = f"./local/partitionned_data/BDLE_10K/partition_{partition}.parquet"
    df_partition.to_parquet(file_path, index=False, engine='pyarrow')